# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:

%load_ext dotenv
%dotenv


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [2]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")



In [3]:
PRICE_DATA

'../../05_src/data/prices/'

In [4]:
import dask.dataframe as dd
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
import dask.dataframe as dd

dd_shift = dd_px.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

C:\Users\dkale\AppData\Local\Temp\ipykernel_22388\3224225999.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby("Ticker", group_keys=False).apply(


In [6]:
dd_returns = dd_shift.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
)

In [11]:
dd_feat = dd_shift.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)
dd_feat

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [24]:
import pandas as pd
pd_feat = dd_feat.compute()
pd_feat

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,returns,hi_lo_range
Ticker,,,,,,,,,,,
DOV,2005-01-03,NaN,18.734226,19.157316,18.693281,19.075428,762305.0,2005,NaN,NaN,0.464035
DOV,2005-01-04,NaN,18.647783,19.011731,18.597740,18.911645,1330378.0,2005,18.734226,-0.004614,0.413991
DOV,2005-01-05,NaN,18.283844,18.857063,18.261098,18.647793,1176575.0,2005,18.647783,-0.019516,0.595965
DOV,2005-01-06,NaN,18.110966,18.311136,18.070022,18.311136,1395122.0,2005,18.283844,-0.009455,0.241114
DOV,2005-01-07,NaN,17.847113,18.138271,17.833464,18.070031,1192984.0,2005,18.110966,-0.014569,0.304807
...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2012-12-24,NaN,NaN,NaN,NaN,NaN,NaN,2012,NaN,NaN,NaN
CTLT,2012-12-26,NaN,NaN,NaN,NaN,NaN,NaN,2012,NaN,NaN,NaN
CTLT,2012-12-27,NaN,NaN,NaN,NaN,NaN,NaN,2012,NaN,NaN,NaN


In [30]:
pd_mva_ret = pd_feat.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(mva_ret = x['returns'].rolling(10).mean())
)

In [38]:
pd_mva_ret.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,returns,hi_lo_range,mva_returns,mva_ret
Ticker,,,,,,,,,,,,,
DOV,2005-01-03,NaN,18.734226,19.157316,18.693281,19.075428,762305.0,2005,NaN,NaN,0.464035,NaN,NaN
DOV,2005-01-04,NaN,18.647783,19.011731,18.597740,18.911645,1330378.0,2005,18.734226,-0.004614,0.413991,NaN,NaN
DOV,2005-01-05,NaN,18.283844,18.857063,18.261098,18.647793,1176575.0,2005,18.647783,-0.019516,0.595965,NaN,NaN
DOV,2005-01-06,NaN,18.110966,18.311136,18.070022,18.311136,1395122.0,2005,18.283844,-0.009455,0.241114,NaN,NaN
DOV,2005-01-07,NaN,17.847113,18.138271,17.833464,18.070031,1192984.0,2005,18.110966,-0.014569,0.304807,NaN,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Answer : Not necessary
+ Would it have been better to do it in Dask? Why?
Answer : Yes, using dask would have been better since it can handle large datasets efficiently.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.